# Let's see where this get us

In [44]:
import numpy as np
import sklearn as skl
import pandas as pd
import sklearn.preprocessing
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

Vitals_labels = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
Test_labels = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

Vitals = ['RRate', 'ABPm', 'SpO2', 'Heartrate']
Test = ['BaseExcess', 'Fibrinogen', 'AST', 'Alkalinephos', 'Bilirubin_total',
         'Lactate', 'TroponinI', 'SaO2',
         'Bilirubin_direct', 'EtCO2']

To generate the final table. Values must be binary. cutn ass mutherfocker
use the 12 entries of heartrate to regress to the label asked. try linear first then go poly. ignore drugs for now

## TASK 1

In [ ]:
df = pd.read_csv("train_features.csv")
df_train= df.groupby(['pid'],sort=False).mean()
df_train_test = df_train[Test]
df_train_vitals=df_train[Vitals]
df_final=pd.concat([df_train_test,df_train_vitals],axis=1)
#df_final = df_final.fillna(0)
#print(df_final)
df_final = df_final.notnull().astype('int')
#print(df_final)

## TASK 3

In [ ]:
labels = pd.read_csv("train_labels.csv")
labels_hr = labels[["LABEL_Heartrate"]].to_numpy()
labels_spo2 = labels[["LABEL_SpO2"]].to_numpy()
labels_abpm = labels[["LABEL_ABPm"]].to_numpy()
labels_rrate = labels[["LABEL_RRate"]].to_numpy()
#scaler0 = skl.preprocessing.StandardScaler()
#scaler0.fit(labels)
#labels = scaler0.transform(labels)
heartrate = df[["Heartrate"]].to_numpy()

heartrate = heartrate.reshape((18995,12))
#print(heartrate[1,:])

#row_mean = np.nanmean(heartrate,axis=1)
#inds = np.where(np.isnan(heartrate))
#heartrate[inds] = np.take(row_mean,inds[1])
#print(heartrate[1,:])

#either imputer is fucked or the data is actually missing an entire patient. 
#imp = SimpleImputer(missing_values=np.nan,strategy='mean')
#heartrate_imp = imp.fit_transform(heartrate)
#print(heartrate_imp[1,:])

#lets find what stupid patient doesnt have a heartrate


#cause some fuckers didnt get a measurement.
heartrate = np.nan_to_num(heartrate,nan=0)

#lets use concatenated full data set TRAIN DATA
fit_df = df_train.fillna(0).to_numpy()
fit_df = np.delete(fit_df,0,1) #remove patient ID
fit_df = np.delete(fit_df,0,1) #remove time spent in ICU
#fit_df = fit_df.reshape((18995,432))
#fit_df = df_train.fillna(0).to_numpy()
scaler1 = skl.preprocessing.StandardScaler()
scaler1.fit(fit_df)
fit_df = scaler1.transform(fit_df)
scaler2 = skl.preprocessing.StandardScaler()
scaler2.fit(heartrate)
heartrate = scaler2.transform(heartrate)
fit_df = np.concatenate((fit_df, heartrate), axis=1)

#SAME FOR TEST DATA
test_df_full = pd.read_csv("test_features.csv")
test_df = test_df_full.groupby(['pid'],sort=False).mean()
test_df = test_df.fillna(0).to_numpy()
test_df = np.delete(test_df,0,1)
test_df = np.delete(test_df,0,1)
labels_test = pd.read_csv("test_features.csv")
heartrate = test_df_full[["Heartrate"]].fillna(0).to_numpy()
heartrate = heartrate.reshape((12664,12))
test_df = scaler1.transform(test_df)
heartrate = scaler2.transform(heartrate)
test_df = np.concatenate((test_df,heartrate), axis=1)

lets try and remove some outliers from the dataset: Top Res: 8.679543614001695

In [47]:
data_mean, data_std = np.mean(fit_df), np.std(fit_df)

#using isolationforest
#iso = skl.ensemble.IsolationForest(contamination=0.1)
#yhat = iso.fit_predict(fit_df)

#using localoutliers
lof = skl.neighbors.LocalOutlierFactor(n_neighbors= 100) #try out different factors
yhat = lof.fit_predict(fit_df)

mask = yhat != -1
fit_df, labels_hr, labels_spo2, labels_abpm, labels_rrate = fit_df[mask,:], labels_hr[mask], labels_spo2[mask], labels_abpm[mask], labels_rrate[mask]
print(fit_df.shape)

(16242, 46)


In [48]:
clf = skl.linear_model.RidgeCV(alphas=[0.01, 0.01, 0.1, 1, 10, 100, 1000, 2000,5000], cv=15)

now let's predict some stuff :D
poly = 18
linearRegression = 9
never fucking use linear svr. takes fucking forever

In [49]:
#predict heart rate:
clf.fit(fit_df,labels_hr)
predict_train = clf.predict(fit_df)
print(np.sqrt(np.mean((predict_train-labels_hr)**2)))
predict_heartrate = clf.predict(test_df)

#predict abpm:
clf.fit(fit_df,labels_abpm)
predict_abpm = clf.predict(test_df)

#predict rrate:
clf.fit(fit_df,labels_rrate)
predict_rrate = clf.predict(test_df)

#predict spo2:
clf.fit(fit_df,labels_spo2)
predict_sp02 = clf.predict(test_df)

8.679543614001695


For some reason linear regression works the best. but i still dont think it's acurate enough

In [55]:
df_task3 = pd.DataFrame({'LABEL_RRate': predict_rrate[:,0], 'LABEL_ABPm': predict_abpm[:,0], 'LABEL_SpO2': predict_sp02[:,0],'LABEL_Heartrate': predict_heartrate[:,0]})
print(df_task3.head)

#print(fit_df)
#print(labels-predict_train)
#print(predict_train)
#print(predict_heartrate)

<bound method NDFrame.head of        LABEL_RRate  LABEL_ABPm  LABEL_SpO2  LABEL_Heartrate
0        14.333795   83.905311   99.043965        82.538233
1        17.476799   90.386710   95.176147        99.628204
2        18.506313   80.201684   98.027821        89.627251
3        16.780357   74.610150   95.877794        87.101032
4        19.143407   75.755701   96.133049        60.976440
...            ...         ...         ...              ...
12659    20.294856   72.905244   95.663825       104.968726
12660    18.093650   92.022292   98.807957        71.872215
12661    19.469431   64.821071   96.750811        83.723222
12662    16.408261   90.412289   98.276552        98.573208
12663    18.072408   78.384271   98.334418        87.719864

[12664 rows x 4 columns]>
[[82.53823316]
 [99.62820398]
 [89.62725121]
 ...
 [83.7232216 ]
 [98.57320826]
 [87.71986383]]


## Final Concatenate

In [ ]:
#final_frickin_df = pd.concat([df_task1, df_task2, df_task3], axis=1)
#compression_opts = dict(method='zip', archive_name='result.csv')
#final_frickin_df.to_csv('result.zip', index=True, compression=compression_opts)